In [1]:
from pointnet.train import setup as train_setup
from pointnet.test import setup as test_setup
from pointnet.utils.train_utils import CheckpointSaver, set_seeds

from cnn import models, utils
from cnn.datasets import *

from easydict import EasyDict
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import numpy as np
import timm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter




In [2]:
class PN_CNN_ENSEMBLE(nn.Module) :
    def __init__(self,pn, cnn) :
        super().__init__()
        
        self.pn = pn
        self.cnn = cnn
        self.cnn_flatten = nn.Sequential(            
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(1)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(2304, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 10)
        )
        
    def forward(self, pn_input, cnn_input) :
        pn_output = self.pn(pn_input)
        cnn_output = self.cnn_flatten(self.cnn.model.forward_features(cnn_input))
        output = self.classifier(torch.concat([pn_output,cnn_output], dim=1))
        
        return output

In [3]:
def validation(model, criterion, 
               cnn_valid_loader, pn_valid_loader,
               device, log_writter):
    
    model.eval()
    true_labels = []
    model_preds = []
    val_loss = []
    with torch.no_grad():
        for (pn, label), (img, _) in tqdm(zip(pn_valid_loader, cnn_valid_loader)):
            label = label.to(device)
            output = model(pn.to(device), img.to(device))

            loss = criterion(output, label)

            val_loss.append(loss.item())

            model_preds += output.argmax(1).detach().cpu().numpy().tolist()
            true_labels += label.detach().cpu().numpy().tolist()            
            
    return np.mean(val_loss), accuracy_score(true_labels, model_preds)

In [4]:
def training(model, optimizer, criterion, 
             cnn_train_loader, cnn_valid_loader, 
             pn_train_loader, pn_valid_loader,
             device) :
        
    # tensorboard
    log_writter = SummaryWriter(train_arg['LOG'])
    best_score = 0
    for epoch in range(1, train_arg['EPOCHS'] + 1) :
        
        tqdm_train = tqdm(zip(pn_train_loader, cnn_train_loader))
        train_acc, train_loss = [], []
        
        for batch, ((pn, label), (img, _)) in enumerate(tqdm_train, start=1) :
            model.train()
            optimizer.zero_grad()
                        
            label = label.to(device)
            output = model(pn.to(device), img.to(device))
            
            loss = criterion(output, label)
            
            loss.backward()
            optimizer.step()
            
            acc = utils.score(label, output)
            train_acc.append(acc)
            train_loss.append(loss.item())
            
            tqdm_train.set_postfix({
                'Training Acc' : np.mean(train_acc),
                'Training Loss' : np.mean(train_loss)
            })
            
            data = {
                'training loss' : loss.item(),
                'training acc' : acc
            }
            utils.logging(log_writter, data, epoch * len(pn_train_loader) + batch)
        
        # validation
        val_loss, val_acc = validation(model, criterion, 
                                       cnn_valid_loader, pn_valid_loader,
                                       device, log_writter)
        
        data = {
            'validation loss' : val_loss,
            'validation acc' : val_acc
        }
        utils.logging(log_writter, data, epoch)
        print(f'Epoch : [{epoch}] Val Loss : [{val_loss}] Val ACC : [{val_acc}]')
        
        if best_score < val_acc:
            best_score = val_acc
            torch.save({
                    "epoch" : epoch,
                    "model_state_dict" : model.state_dict(),
                    "optimizer_state_dict" : optimizer.state_dict()
                }, './ckpt/'+str(epoch)+'E-val'+str(best_score)+'-'+train_arg['output'])

In [5]:
pn_arg = EasyDict({
    "config" : "./pointnet/configs/pointnet2_config.yaml",
    "workspace" : './pointnet/workspace',
    "tensorboard_dir" : "./pointnet/runs",
    'restore_path' : './pointnet/workspace/pointnet2/checkpoints/best.epoch0099-score0.9676.pth',
    "write_lr" : "n",
    "seed" : 2455,
    "save_step" : 10,
    "continue_train" : "y",
})

train_arg = EasyDict({
    'EPOCHS':5,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':16,
    
    'output' : 'pointnet_effib0_ensemble.pth',
    'LOG' : "./tensorboard/pn_cnn",   
})
device = torch.device('cuda')

In [6]:
_, _, _, dataloaders, _, pn_model, _, _, _, _, _, _, _, _ =train_setup(pn_arg)

train data: 40000
val data: 10000
loading model from ./pointnet/workspace/pointnet2/checkpoints/best.epoch0099-score0.9676.pth...
Error(s) in loading state_dict for PointNet2Cls:
	Unexpected key(s) in state_dict: "classifier.0.weight", "classifier.0.bias", "classifier.1.weight", "classifier.1.bias", "classifier.1.running_mean", "classifier.1.running_var", "classifier.1.num_batches_tracked", "classifier.4.weight", "classifier.4.bias", "classifier.5.weight", "classifier.5.bias", "classifier.5.running_mean", "classifier.5.running_var", "classifier.5.num_batches_tracked", "classifier.8.weight", "classifier.8.bias". 
ignore unmatching keys...
config path: ./pointnet/configs/pointnet2_config.yaml


In [7]:
cnn_model = models.CNN('efficientnet_b0').to(device)
checkpoint = torch.load('./ckpt/69E-val0.9611-4fold_3fold_2fold_1fold_0fold_scratch-weigt_freeze10E-mixup25E-grid_shuffle35E-focal-effib0.pth')
cnn_model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [8]:
pn_train_loader = dataloaders['train']
pn_valid_loader = dataloaders['val']

In [9]:
img_set, label_set, transform = image_label_dataset(df_path='./data/train.csv', 
                                                     img_path = './data/img/224img_train/*',
                                                     div=0.8, grid_shuffle_p=0, training=True)
cnn_train_loader, cnn_valid_loader = train_and_valid_dataload(img_set, label_set, transform, batch_size=16)

In [10]:
pn_model = utils.weight_freeze(pn_model)
cnn_model = utils.weight_freeze(cnn_model)
ensemble_model = PN_CNN_ENSEMBLE(pn_model, cnn_model).to(device)

optimizer = torch.optim.Adam(ensemble_model.parameters(), 
                             lr=train_arg['LEARNING_RATE'])
criterion = nn.CrossEntropyLoss() #FocalLoss(CFG['focal_alpha'], CFG['focal_gamma'])

In [11]:
training(ensemble_model, optimizer, criterion, 
         cnn_train_loader, cnn_valid_loader, 
         pn_train_loader, pn_valid_loader,
         device)

2500it [18:15,  2.28it/s, Training Acc=0.961, Training Loss=0.179]
625it [04:26,  2.35it/s]


Epoch : [1] Val Loss : [0.06749554616883396] Val ACC : [0.9798]


2500it [17:34,  2.37it/s, Training Acc=0.975, Training Loss=0.0876]
625it [04:13,  2.46it/s]


Epoch : [2] Val Loss : [0.05453034588862211] Val ACC : [0.9837]


2500it [17:28,  2.38it/s, Training Acc=0.979, Training Loss=0.0709]
625it [04:13,  2.46it/s]


Epoch : [3] Val Loss : [0.05328896128199995] Val ACC : [0.9842]


2500it [17:32,  2.38it/s, Training Acc=0.98, Training Loss=0.0627] 
625it [04:26,  2.35it/s]


Epoch : [4] Val Loss : [0.0559958429697901] Val ACC : [0.984]


2500it [19:29,  2.14it/s, Training Acc=0.983, Training Loss=0.0558]
625it [04:43,  2.20it/s]

Epoch : [5] Val Loss : [0.05118099400047213] Val ACC : [0.9851]


In [3]:
def predict(model, pn_loader, cnn_loader, device):
    model.to(device)
    model.eval()
    model_preds = []
    with torch.no_grad():
        for (pn, _), img in tqdm(zip(pn_loader, cnn_loader)):
            
            output = model(pn.to(device), img.to(device))
            
            model_preds += output.argmax(1).detach().cpu().numpy().tolist()
    
    return model_preds

In [4]:
pn_arg = EasyDict({
    "batch_size" : 16,
    "csv_filename" : "5E-val0.9851-pointnet_effib0_ensemble",
    "config" : "./pointnet/configs/pointnet2_config.yaml",
    "workspace" : './pointnet/workspace',
    "seed" : None,
})

test_arg = EasyDict({
    'batch_size' : 16,
})
device = torch.device('cuda')

In [5]:
_, pn_loader, pn_model, _, _ = test_setup(pn_arg)
cnn_model = models.CNN('efficientnet_b0').to(device)


test data: 40000
loading model from ./pointnet/workspace\pointnet2\checkpoints\best.epoch0099-score0.9676.pth...
Error(s) in loading state_dict for PointNet2Cls:
	Unexpected key(s) in state_dict: "classifier.0.weight", "classifier.0.bias", "classifier.1.weight", "classifier.1.bias", "classifier.1.running_mean", "classifier.1.running_var", "classifier.1.num_batches_tracked", "classifier.4.weight", "classifier.4.bias", "classifier.5.weight", "classifier.5.bias", "classifier.5.running_mean", "classifier.5.running_var", "classifier.5.num_batches_tracked", "classifier.8.weight", "classifier.8.bias". 
ignore unmatching keys...


In [6]:
checkpoint = torch.load('./ckpt/5E-val0.9851-pointnet_effib0_ensemble.pth')

ensemble_model = PN_CNN_ENSEMBLE(pn_model, cnn_model).to(device)
ensemble_model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [7]:
test_img = img_parser('./data/img/224img_test/*', None, training=False)
transform = transform_parser(grid_shuffle_p=0)
cnn_loader = custom_dataload(test_img, None, test_arg.batch_size, transform, False)

In [8]:
preds = predict(ensemble_model, pn_loader, cnn_loader, device)

2500it [18:09,  2.29it/s]


In [9]:
test_df = pd.read_csv('./data/sample_submission.csv')
test_df['label'] = preds
test_df.to_csv('./submission/5E-val0.9851-pointnet_effib0_ensemble.csv', index=False)